** 스마트워치 센서 데이터 기반 행동 분류 3일차 **
# --------- 과제5 : 모델 검증 ---------

### [미션0] 과제 이해하기
   * 그동안 XGBoost 알고리즘을 튜닝해 train 데이터셋에 대한 최고의 행동패턴 분류 모델을 만들었습니다.
   * 이제 test 데이터셋을 사용하여 그동안 만들고 튜닝해온 모델에 데이터를 학습시켜 정확도를 확인 하겠습니다.
   * 미션4에서 튜닝한 XGBoost 모델의 과대적합/과소적합 여부를 확인하고 문제를 해소해 보세요.
   

※  과제완수 조건: train 데이터셋으로 예측한 결과와 test 데이터셋으로 예측한 결과를 비교하여 과대적합/과소적합 여부를 판단하고 과대적합/과소적합을 해결해 주세요.  

---

### [미션1] 데이터 불러오기


* pandas, numpy, matplotlib, seaborn, os 등 데이터 분석에필요한 라이브러리와 모듈을 불러오세요.
* 라이브러리가 없을땐 !pip 명령어를 통해 라이브러리를 설치 하세요.
* 현위치 하위 'data' 폴더에서 2일차에 생성한 파일들을 읽어 오세요.

	- 실습순서: 라이브러리 불러오기 -> 모델 불러오기 -> 결과 데이터 불러오기 -> test 데이터 불러오기 -> test 데이터 나누기

In [23]:
# pandas, numpy, matplotlib, seaborn, os 등 필요 라이브러리 호출
# 데이터를 나누기 위한 sklearn.model_selection 모듈의 train_test_split 함수 사용
# 모델 성능 평가 출력을 위해 sklearn.metrics 모듈의 모든 클래스 사용
# XGBClassifier 알고리즘 사용을 위한 모듈 추가
# 저장한 모델을 불러오기 위한 joblib 라이브러리 추가
## 그외 라이브러리는 필요에 따라 호출 하세요.

import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sns
import os
import joblib 

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from xgboost import XGBClassifier


In [24]:
# 2일차 데이터 불러오기
## 2일차에 만들었던 최고 성능의 모델(hpt_xgb_top_model.pkl)을 불러와 hpt_xgb_top_model 변수에 할당 합니다.
## 모델 결과에 가장 영향을 많이 미친 feature의 이름이 담긴 importance_top 데이터(importance_top.pkl)를 불러와 importance_top 변수에 할당 합니다.

hpt_xgb_top_model = joblib.load('hpt_xgb_top_model.pkl')
importance_top = joblib.load('importance_top.pkl')



In [25]:
# 2일차 모델 결과 데이터 불러오기
## 모델별 정확도를 비교 할 수 있도록 결과 저장 파일을 불러 옵니다.
## 'data' 폴더에서 2일차에 생성한 result4.csv 파일을 불러와서 result 변수에 할당 합니다.
## result 변수에 할당이 잘 되었는지 확인해 주세요. 


result = pd.read_csv('./data/result4.csv')
result

model_name valid_data  accuracy_score  f1_score
0          svc      train        0.968839  0.971715
1           lr      train        0.983003  0.984583
2          knn      train        0.954674  0.958345
3          xgb      train        0.988669  0.989470
4      hpt_xgb      train        0.990368  0.991052
5  hpt_xgb_150      train        0.990935  0.991562
6   hpt_xgb_50      train        0.985269  0.985278
7  hpt_xgb_top      train        0.991501  0.992040

In [26]:
# test 데이터 불러오기
## 'data' 폴더에서 test_data.csv 파일을 불러와서 test_data 변수에 할당해 주세요.
## 잘 할당 되었는지 데이터를 확인해 주세요.

test_data = pd.read_csv('./data/test_data.csv')
test_data




tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0              0.284379          -0.021981          -0.116683   
1              0.277440          -0.028086          -0.118412   
2              0.305833          -0.041023          -0.087303   
3              0.276053          -0.016487          -0.108381   
4              0.271998           0.016904          -0.078856   
...                 ...                ...                ...   
1466           0.278725          -0.015262          -0.106398   
1467           0.275803          -0.019257          -0.109078   
1468           0.240402           0.006361          -0.121377   
1469           0.135873          -0.020675          -0.116644   
1470           0.241563          -0.020145          -0.083858   

      tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  \
0            -0.992490         -0.979640         -0.963321         -0.992563   
1            -0.996620         -0.927676         -0.972294         -0.997346   
2             0.006880          0.182800         -0.237984          0.005642   
3            -0.995379         -0.983978         -0.975854         -0.995877   
4            -0.973468         -0.702462         -0.869450         -0.979810   
...                ...               ...               ...               ...   
1466         -0.993625         -0.982845         -0.974745         -0.993963   
1467         -0.998614         -0.991621         -0.987403         -0.998813   
1468         -0.045805          0.189930          0.332664         -0.114706   
1469         -0.960526         -0.955134         -0.985818         -0.963115   
1470          0.410202          0.260990         -0.272913          0.345287   

      tBodyAcc-mad()-Y  tBodyAcc-mad()-Z  tBodyAcc-max()-X  ...  \
0            -0.977304         -0.958142         -0.938850  ...   
1            -0.931405         -0.971788         -0.939837  ...   
2             0.028616         -0.236474          0.016311  ...   
3            -0.985280         -0.974907         -0.941425  ...   
4            -0.711601         -0.856807         -0.920760  ...   
...                ...               ...               ...  ...   
1466         -0.981100         -0.974596         -0.939303  ...   
1467         -0.991503         -0.986802         -0.945442  ...   
1468          0.157771          0.195271          0.210139  ...   
1469         -0.971338         -0.988261         -0.946289  ...   
1470          0.196325         -0.300247          0.622167  ...   

      fBodyBodyGyroJerkMag-kurtosis()  angle(tBodyAccMean,gravity)  \
0                           -0.850065                    -0.018043   
1                           -0.613367                    -0.022456   
2                            0.394388                    -0.362616   
3                           -0.841455                     0.289548   
4                            0.214219                     0.010111   
...                               ...                          ...   
1466                        -0.874066                    -0.054788   
1467                        -0.721050                     0.076333   
1468                        -0.615554                     0.330378   
1469                        -0.422383                    -0.048474   
1470                        -0.843214                     0.374865   

      angle(tBodyAccJerkMean),gravityMean)  angle(tBodyGyroMean,gravityMean)  \
0                                 0.092304                          0.074220   
1                                -0.155414                          0.247498   
2                                 0.171069                          0.576349   
3                                 0.079801                         -0.020033   
4                                 0.114179                         -0.830776   
...                                    ...                               ...   
1466                              0.712975                

In [27]:
# test X, Y 나누기
## 모델을 검증 하기위해 test 데이터를 사용하여 test_x , test_y_map(target) 데이터를 생성 합니다.
## test_x 에는 test_data 데이터셋에서 2일차에 최고의 성능을 내는 feature 들만(importance_top) 분리하여 할당 합니다.  
## test_y 에는 모델의 예측 결과와 비교할 수 있도록 test_data['Activity'] 열의 값을 숫자형으로 치환하여 할당 합니다.
## 치환 데이터: 'STANDING':0, 'SITTING':1, 'LAYING':2, 'WALKING':3, 'WALKING_UPSTAIRS':4, 'WALKING_DOWNSTAIRS':5


test_x = test_data[importance_top]
test_y_map = test_data['Activity'].map({'STANDING':0, 'SITTING':1, 'LAYING':2, 'WALKING':3, 'WALKING_UPSTAIRS':4, 'WALKING_DOWNSTAIRS':5})




### [미션2] 모델 검증하기


* test 데이터를 사용하여 2일차에 튜닝한 모델을 검증하고 과대적합/과소적합을 판단 합니다.


	- 실습순서: test 데이터 결과 예측 -> 모델 성능 평가 -> 예측결과 저장 및 비교 -> 과대적합/과소적합 판단

In [28]:
# test 데이터 결과 예측
# 2일차에 생성한 hpt_xgb_top_model 에 text_x 데이터를 적용한 예측 결과를 test_hpt_xgb_top_pred 변수에 할당 합니다.
# 사용할 모델은 .pkl 파일에서 로드한 hpt_xgb_top_model 변수를 사용 합니다.

test_hpt_xgb_top_pred = hpt_xgb_top_model.predict(test_x)
test_hpt_xgb_top_pred


array([1, 0, 3, ..., 5, 2, 5], dtype=int64)

In [29]:
# test 데이터 적용 모델 성능 평가 하기
## 모델이 결과를 잘 예측하였는지 3가지 성능지표로 결과를 평가 합니다.
## 3가지 성능지표: accuracy_score, confusion_matrix, classification_report

print('accuracy_score: ',accuracy_score(test_y_map,test_hpt_xgb_top_pred))
print('\n confusion_matrix: \n',confusion_matrix(test_y_map,test_hpt_xgb_top_pred))
print('\n classification_report: \n',classification_report(test_y_map,test_hpt_xgb_top_pred))

accuracy_score:  0.9857239972807614

 confusion_matrix: 
 [[282   5   0   0   0   0]
 [  9 245   0   0   0   0]
 [  0   2 290   0   0   0]
 [  0   0   0 225   3   0]
 [  0   0   0   0 215   0]
 [  0   0   0   2   0 193]]

 classification_report: 
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       287
           1       0.97      0.96      0.97       254
           2       1.00      0.99      1.00       292
           3       0.99      0.99      0.99       228
           4       0.99      1.00      0.99       215
           5       1.00      0.99      0.99       195

    accuracy                           0.99      1471
   macro avg       0.99      0.99      0.99      1471
weighted avg       0.99      0.99      0.99      1471



In [30]:
# hpt_xgb_top_model 예측결과 저장
## result 데이터프레임 9번 인덱스에 'test_hpt_xgb_top'(모델명), 'test'(검증 데이터명), accuracy_score 결과, f1_score 결과 를 할당해 주세요.
## f1_score의 평균은 'macro' 파라미터를 사용 합니다.
## 예측 결과가 잘 할당 되었는지 확인해 주세요.


result.loc[9] = ['test_hpt_xgb_top', 'test', accuracy_score(test_y_map,test_hpt_xgb_top_pred), f1_score(test_y_map,test_hpt_xgb_top_pred, average = 'macro')]
result



model_name valid_data  accuracy_score  f1_score
0               svc      train        0.968839  0.971715
1                lr      train        0.983003  0.984583
2               knn      train        0.954674  0.958345
3               xgb      train        0.988669  0.989470
4           hpt_xgb      train        0.990368  0.991052
5       hpt_xgb_150      train        0.990935  0.991562
6        hpt_xgb_50      train        0.985269  0.985278
7       hpt_xgb_top      train        0.991501  0.992040
9  test_hpt_xgb_top       test        0.985724  0.986275

\# 과대적합/과소적합 판단<BR>
* 8번 hpt_xgb_top 결과와 9번 test_hpt_xgb_top 결과를 보고<BR>
* 과소적합 또는 과대적합 여부를 판단 합니다.<BR>

### [미션3] 과대적합 해소하기

* 과대적합은 train 데이터의 예측 결과가 test 데이터의 예측 결과보다 정확도가 높아 모델이 test 데이터 보다 train 데이터에 보다 fit 하게 학습 되었을때 나타납니다.
* 과대적합이 발생할 경우 데이터 별로 다른 정확도를 보일 수 있어, 어느 데이터나 비슷한 정확도를 얻기위해 과대적합을 해소해 주어야 합니다.
* 과대적합을 해소하는 방법으로 데이터의 양을 늘려 모델에 학습량을 늘려주거나, 모델을 단순하게 만드는 방법이 있습니다.

#### ① 과대적합 해소하기: 데이터 학습량 늘리기


    - 실습순서: train 데이터 불러오기 -> train 데이터 + test 데이터 합치기 -> 인덱스 초기화 -> X, Y 데이터 나누기 -> 학습 데이터 나누기 -> 모델링 및 결과예측 -> 모델 성능 평가 -> 예측결과 저장 및 비교

In [31]:
# train 데이터 불러오기
## 'data' 폴더에서 'train_data.csv' 파일을 불러와서 train_data 변수에 할당 합니다.
## 잘 불러와 졌는지 데이터를 확인 합니다.

train_data = pd.read_csv('./data/train_data.csv')
train_data

tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0              0.288508          -0.009196          -0.103362   
1              0.265757          -0.016576          -0.098163   
2              0.278709          -0.014511          -0.108717   
3              0.289795          -0.035536          -0.150354   
4              0.394807           0.034098           0.091229   
...                 ...                ...                ...   
5876           0.277194          -0.012389          -0.131974   
5877           0.191568           0.013328          -0.105174   
5878           0.267981          -0.018348          -0.107440   
5879           0.212787          -0.048130          -0.121001   
5880           0.283815          -0.040658          -0.138018   

      tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  \
0            -0.988986         -0.962797         -0.967422         -0.989000   
1            -0.989551         -0.994636         -0.987435         -0.990189   
2            -0.997720         -0.981088         -0.994008         -0.997934   
3            -0.231727         -0.006412         -0.338117         -0.273557   
4             0.088489         -0.106636         -0.388502         -0.010469   
...                ...               ...               ...               ...   
5876         -0.994046         -0.940578         -0.917337         -0.994261   
5877         -0.126969         -0.121729         -0.327480         -0.192523   
5878         -0.991303         -0.989881         -0.990313         -0.992386   
5879         -0.041373          0.052449         -0.585361         -0.100714   
5880         -0.992662         -0.747190         -0.920248         -0.993410   

      tBodyAcc-mad()-Y  tBodyAcc-mad()-Z  tBodyAcc-max()-X  ...  \
0            -0.962596         -0.965650         -0.929747  ...   
1            -0.993870         -0.987558         -0.937337  ...   
2            -0.982187         -0.995017         -0.942584  ...   
3             0.014245         -0.347916          0.008288  ...   
4            -0.109680         -0.346372          0.584131  ...   
...                ...               ...               ...  ...   
5876         -0.932830         -0.908088         -0.936219  ...   
5877         -0.109923         -0.295286          0.078644  ...   
5878         -0.988852         -0.991237         -0.936099  ...   
5879          0.023353         -0.554707          0.219814  ...   
5880         -0.768055         -0.924587         -0.935871  ...   

      fBodyBodyGyroJerkMag-kurtosis()  angle(tBodyAccMean,gravity)  \
0                           -0.816696                    -0.042494   
1                           -0.693515                    -0.062899   
2                           -0.829311                     0.000265   
3                           -0.408956                    -0.255125   
4                           -0.563437                    -0.044344   
...                               ...                          ...   
5876                        -0.690363                    -0.034888   
5877                        -0.879215                     0.721718   
5878                        -0.886851                     0.060173   
5879                        -0.053556                     0.260880   
5880                        -0.696584                     0.107690   

      angle(tBodyAccJerkMean),gravityMean)  angle(tBodyGyroMean,gravityMean)  \
0                                -0.044218                          0.307873   
1                                 0.388459                         -0.765014   
2                                -0.525022                         -0.891875   
3                                 0.612804                          0.747381   
4                                -0.845268                         -0.974650   
...                                    ...                               ...   
5876                             -0.261437                

In [32]:
# 데이터 합치기
## train 데이터셋과 test 데이셋을 합쳐 데이터의 양을 늘려 줍니다.
## train 데이터셋과 test 데이터셋을 위/아래로 합쳐서 all_data 변수에 할당 합니다.
## 참고함수: concat


all_data = pd.concat([train_data, test_data])
all_data

tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0              0.288508          -0.009196          -0.103362   
1              0.265757          -0.016576          -0.098163   
2              0.278709          -0.014511          -0.108717   
3              0.289795          -0.035536          -0.150354   
4              0.394807           0.034098           0.091229   
...                 ...                ...                ...   
1466           0.278725          -0.015262          -0.106398   
1467           0.275803          -0.019257          -0.109078   
1468           0.240402           0.006361          -0.121377   
1469           0.135873          -0.020675          -0.116644   
1470           0.241563          -0.020145          -0.083858   

      tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  \
0            -0.988986         -0.962797         -0.967422         -0.989000   
1            -0.989551         -0.994636         -0.987435         -0.990189   
2            -0.997720         -0.981088         -0.994008         -0.997934   
3            -0.231727         -0.006412         -0.338117         -0.273557   
4             0.088489         -0.106636         -0.388502         -0.010469   
...                ...               ...               ...               ...   
1466         -0.993625         -0.982845         -0.974745         -0.993963   
1467         -0.998614         -0.991621         -0.987403         -0.998813   
1468         -0.045805          0.189930          0.332664         -0.114706   
1469         -0.960526         -0.955134         -0.985818         -0.963115   
1470          0.410202          0.260990         -0.272913          0.345287   

      tBodyAcc-mad()-Y  tBodyAcc-mad()-Z  tBodyAcc-max()-X  ...  \
0            -0.962596         -0.965650         -0.929747  ...   
1            -0.993870         -0.987558         -0.937337  ...   
2            -0.982187         -0.995017         -0.942584  ...   
3             0.014245         -0.347916          0.008288  ...   
4            -0.109680         -0.346372          0.584131  ...   
...                ...               ...               ...  ...   
1466         -0.981100         -0.974596         -0.939303  ...   
1467         -0.991503         -0.986802         -0.945442  ...   
1468          0.157771          0.195271          0.210139  ...   
1469         -0.971338         -0.988261         -0.946289  ...   
1470          0.196325         -0.300247          0.622167  ...   

      fBodyBodyGyroJerkMag-kurtosis()  angle(tBodyAccMean,gravity)  \
0                           -0.816696                    -0.042494   
1                           -0.693515                    -0.062899   
2                           -0.829311                     0.000265   
3                           -0.408956                    -0.255125   
4                           -0.563437                    -0.044344   
...                               ...                          ...   
1466                        -0.874066                    -0.054788   
1467                        -0.721050                     0.076333   
1468                        -0.615554                     0.330378   
1469                        -0.422383                    -0.048474   
1470                        -0.843214                     0.374865   

      angle(tBodyAccJerkMean),gravityMean)  angle(tBodyGyroMean,gravityMean)  \
0                                -0.044218                          0.307873   
1                                 0.388459                         -0.765014   
2                                -0.525022                         -0.891875   
3                                 0.612804                          0.747381   
4                                -0.845268                         -0.974650   
...                                    ...                               ...   
1466                              0.712975                

In [33]:
# 데이터 인덱스 초기화
## 데이터셋을 위아래로 합치면 인덱스 중복이 발생 합니다.
## 인덱스가 중복되지 않도록 인덱스를 리셋해 주세요.

all_data.reset_index(drop=True, inplace=True)
all_data

tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0              0.288508          -0.009196          -0.103362   
1              0.265757          -0.016576          -0.098163   
2              0.278709          -0.014511          -0.108717   
3              0.289795          -0.035536          -0.150354   
4              0.394807           0.034098           0.091229   
...                 ...                ...                ...   
7347           0.278725          -0.015262          -0.106398   
7348           0.275803          -0.019257          -0.109078   
7349           0.240402           0.006361          -0.121377   
7350           0.135873          -0.020675          -0.116644   
7351           0.241563          -0.020145          -0.083858   

      tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  \
0            -0.988986         -0.962797         -0.967422         -0.989000   
1            -0.989551         -0.994636         -0.987435         -0.990189   
2            -0.997720         -0.981088         -0.994008         -0.997934   
3            -0.231727         -0.006412         -0.338117         -0.273557   
4             0.088489         -0.106636         -0.388502         -0.010469   
...                ...               ...               ...               ...   
7347         -0.993625         -0.982845         -0.974745         -0.993963   
7348         -0.998614         -0.991621         -0.987403         -0.998813   
7349         -0.045805          0.189930          0.332664         -0.114706   
7350         -0.960526         -0.955134         -0.985818         -0.963115   
7351          0.410202          0.260990         -0.272913          0.345287   

      tBodyAcc-mad()-Y  tBodyAcc-mad()-Z  tBodyAcc-max()-X  ...  \
0            -0.962596         -0.965650         -0.929747  ...   
1            -0.993870         -0.987558         -0.937337  ...   
2            -0.982187         -0.995017         -0.942584  ...   
3             0.014245         -0.347916          0.008288  ...   
4            -0.109680         -0.346372          0.584131  ...   
...                ...               ...               ...  ...   
7347         -0.981100         -0.974596         -0.939303  ...   
7348         -0.991503         -0.986802         -0.945442  ...   
7349          0.157771          0.195271          0.210139  ...   
7350         -0.971338         -0.988261         -0.946289  ...   
7351          0.196325         -0.300247          0.622167  ...   

      fBodyBodyGyroJerkMag-kurtosis()  angle(tBodyAccMean,gravity)  \
0                           -0.816696                    -0.042494   
1                           -0.693515                    -0.062899   
2                           -0.829311                     0.000265   
3                           -0.408956                    -0.255125   
4                           -0.563437                    -0.044344   
...                               ...                          ...   
7347                        -0.874066                    -0.054788   
7348                        -0.721050                     0.076333   
7349                        -0.615554                     0.330378   
7350                        -0.422383                    -0.048474   
7351                        -0.843214                     0.374865   

      angle(tBodyAccJerkMean),gravityMean)  angle(tBodyGyroMean,gravityMean)  \
0                                -0.044218                          0.307873   
1                                 0.388459                         -0.765014   
2                                -0.525022                         -0.891875   
3                                 0.612804                          0.747381   
4                                -0.845268                         -0.974650   
...                                    ...                               ...   
7347                              0.712975                

In [34]:
# X, Y 데이터 나누기
## 모델 학습을 위해 feature(X) 데이터와 target(Y) 데이터를 나누어 주어야 합니다.
## target 데이터인 all_data['Activity'] 값을 모델이 인식할 수 있게 숫자로 변환하여 all_y_map 변수에 할당 합니다.
## all_data 데이터에서 2일차에서 선정한 중요도가 높은 feature 들만 골라 all_x 변수에 할당 합니다.
## 중요도가 높은 feature 들의 이름은 .pkl 파일에서 불러와 importance_top 변수에 저장되어 있습니다.

all_y_map = all_data['Activity'].map({'STANDING':0, 'SITTING':1, 'LAYING':2, 'WALKING':3, 'WALKING_UPSTAIRS':4, 'WALKING_DOWNSTAIRS':5})
all_x = all_data[importance_top]
all_x

fBodyAccMag-mad()  tBodyGyroJerk-entropy()-X  \
0             -0.980498                  -0.470202   
1             -0.990134                  -0.609813   
2             -0.991574                  -0.410492   
3             -0.489603                   0.651518   
4              0.206229                   0.763908   
...                 ...                        ...   
7347          -0.989536                  -0.713558   
7348          -0.995179                  -0.805478   
7349           0.136070                   0.680693   
7350          -0.947412                  -0.134897   
7351           0.514575                   0.653277   

      fBodyAcc-bandsEnergy()-1,8.2  tBodyGyroJerk-mad()-X  \
0                        -0.998879              -0.986107   
1                        -0.999808              -0.992184   
2                        -0.999967              -0.984787   
3                        -0.822895              -0.363761   
4                        -0.779931              -0.554993   
...                            ...                    ...   
7347                     -0.999165              -0.995914   
7348                     -0.999662              -0.996677   
7349                     -0.387808              -0.338098   
7350                     -0.999704              -0.944816   
7351                     -0.764802              -0.485523   

      tGravityAcc-mean()-X  fBodyAccMag-std()  fBodyAcc-skewness()-X  \
0                 0.875254          -0.981567              -0.199419   
1                -0.134711          -0.992435              -0.235336   
2                 0.965965          -0.994252              -0.650195   
3                 0.927343          -0.575620              -0.151351   
4                 0.901125          -0.049606               0.030987   
...                    ...                ...                    ...   
7347              0.966352          -0.991764              -0.170476   
7348              0.944731          -0.994869              -0.476480   
7349              0.889382          -0.172156              -0.401334   
7350             -0.755438          -0.940337               0.493163   
7351              0.937509           0.124844              -0.167568   

      fBodyBodyAccJerkMag-max()  angle(Y,gravityMean)  \
0                     -0.993207              0.331298   
1                     -0.993308             -0.769186   
2                     -0.993402              0.202434   
3                     -0.316349              0.287154   
4                     -0.352163              0.264952   
...                         ...                   ...   
7347                  -0.993933              0.120184   
7348                  -0.996915              0.212111   
7349                  -0.182928              0.185363   
7350                  -0.950329             -0.474618   
7351                  -0.153873              0.193947   

      tGravityAcc-arCoeff()-Z,1  ...  tBodyGyro-arCoeff()-Y,2  \
0                     -0.386458  ...                 0.359116   
1                      0.021547  ...                 0.298081   
2                      0.075337  ...                -0.087433   
3                     -0.352702  ...                 0.320605   
4                     -0.769572  ...                 0.432295   
...                         ...  ...                      ...   
7347                  -0.501844  ...                -0.098912   
7348                  -0.266155  ...                -0.010259   
7349                  -0.205011  ...                 0.183546   
7350                  -0.099316  ...                -0.044678   
7351                  -0.452315  ...                 0.220875   

      fBodyGyro-meanFreq()-Y  tBodyGyro-arCoeff()-Y,1  \
0                  -0.535663                -0.437778   
1                  -0.446218                -0.473290   
2                   0.402094                 0.304278   
3                  -0.185642                -0.312109   
4       

In [35]:
# 학습 데이터 나누기
## 학습에 필요한 all_x 와 all_y_map 데이터를 학습 7 : 검증 3 비율로 나누어 주세요.
## 이때 random_state 는 2023 으로 설정해 주세요.
## 데이터를 할당받을 변수명: all_x_train, all_x_val, all_y_train, all_y_val


all_x_train, all_x_val, all_y_train, all_y_val = train_test_split(all_x,all_y_map,train_size=0.7,random_state=2023)


In [36]:
# AI 모델링 및 결과 예측
## XGBClassifier 알고리즘을 사용하여 all_hpt_xgb_model 변수에 모델을 생성 및 초기화 합니다.
## 이때 파라미터는 튜닝에서 가장 성능이 좋았던 값으로 지정하고, random_state는 2023 으로 설정해 주세요.
## 모델이 생성되면 all_x_train, all_y_train 로 모델을 학습시켜 줍니다.
## 모델 학습이 완료되면 test_x 데이터의 결과를 예측합니다.

all_hpt_xgb_model = XGBClassifier(learning_rate=0.3, max_depth=3, random_state=2023)
all_hpt_xgb_model.fit(all_x_train, all_y_train)
all_hpt_xgb_pred = all_hpt_xgb_model.predict(test_x)
all_hpt_xgb_pred


array([1, 0, 3, ..., 5, 2, 5], dtype=int64)

In [37]:
# all_hpt_xgb_model 성능 평가 하기
## 모델이 결과를 잘 예측하였는지 3가지 성능지표로 결과를 평가 합니다.
## 3가지 성능지표: accuracy_score, confusion_matrix, classification_report

print('accuracy_score: ',accuracy_score(test_y_map,all_hpt_xgb_pred))
print('\n confusion_matrix: \n',confusion_matrix(test_y_map,all_hpt_xgb_pred))
print('\n classification_report: \n',classification_report(test_y_map,all_hpt_xgb_pred))

accuracy_score:  0.9966009517335146

 confusion_matrix: 
 [[286   1   0   0   0   0]
 [  4 250   0   0   0   0]
 [  0   0 292   0   0   0]
 [  0   0   0 228   0   0]
 [  0   0   0   0 215   0]
 [  0   0   0   0   0 195]]

 classification_report: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       287
           1       1.00      0.98      0.99       254
           2       1.00      1.00      1.00       292
           3       1.00      1.00      1.00       228
           4       1.00      1.00      1.00       215
           5       1.00      1.00      1.00       195

    accuracy                           1.00      1471
   macro avg       1.00      1.00      1.00      1471
weighted avg       1.00      1.00      1.00      1471



In [38]:
# all_hpt_xgb_model 예측결과 저장
## result 데이터프레임 10번 인덱스에 'all_hpt_xgb'(모델명), 'test'(검증 데이터명), accuracy_score 결과, f1_score 결과를 할당해 주세요.
## f1_score의 평균은 'macro' 파라미터를 사용 합니다.
## 예측 결과가 잘 할당 되었는지 확인해 주세요.

result.loc[10] = ['all_hpt_xgb', 'test', accuracy_score(test_y_map,all_hpt_xgb_pred), f1_score(test_y_map,all_hpt_xgb_pred, average = 'macro')]
result


model_name valid_data  accuracy_score  f1_score
0                svc      train        0.968839  0.971715
1                 lr      train        0.983003  0.984583
2                knn      train        0.954674  0.958345
3                xgb      train        0.988669  0.989470
4            hpt_xgb      train        0.990368  0.991052
5        hpt_xgb_150      train        0.990935  0.991562
6         hpt_xgb_50      train        0.985269  0.985278
7        hpt_xgb_top      train        0.991501  0.992040
9   test_hpt_xgb_top       test        0.985724  0.986275
10       all_hpt_xgb       test        0.996601  0.996906

\# 결과 판단<BR>
* 데이터의 학습량을 늘려 과대적합을 해결하려 하였습니다.<BR>
* 결과를 보고 과대적합이 해소 되었는지 확인 합니다.<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
<BR>

#### ② 과대적합 해소하기: 모델 단순화 하기

- 실습순서: 모델 단순화 및 모델링 -> 모델 성능 평가 -> 예측결과 저장 및 비교

In [39]:
# 모델 단순화 및 결과 예측
## XGBClassifier 알고리즘을 사용하여 all_hpt_depth_xgb_model 변수에 모델을 생성 및 초기화 합니다.
## 이때 과대적합이 일어나지 않도록 XGBClassifier의 깊이 파라미터를 조정해 모델을 단순화 하세요.
## random_state는 2023 으로 설정해 주세요.
## 모델이 생성되면 all_x_train, all_y_train 로 모델을 학습시켜 줍니다.
## 모델 학습이 완료되면 test_x 데이터의 결과를 예측합니다.

all_hpt_depth_xgb_model = XGBClassifier(learning_rate=0.3, max_depth=2, random_state=2023)
all_hpt_depth_xgb_model.fit(all_x_train, all_y_train)
all_hpt_depth_xgb_pred = all_hpt_depth_xgb_model.predict(test_x)
all_hpt_depth_xgb_pred


array([1, 0, 3, ..., 5, 2, 5], dtype=int64)

In [40]:
# all_hpt_depth_xgb_model 성능 평가 하기
## 모델이 결과를 잘 예측하였는지 3가지 성능지표로 결과를 평가 합니다.
## 3가지 성능지표: accuracy_score, confusion_matrix, classification_report

print('accuracy_score: ',accuracy_score(test_y_map,all_hpt_depth_xgb_pred))
print('\n confusion_matrix: \n',confusion_matrix(test_y_map,all_hpt_depth_xgb_pred))
print('\n classification_report: \n',classification_report(test_y_map,all_hpt_depth_xgb_pred))

accuracy_score:  0.9959211420802175

 confusion_matrix: 
 [[286   1   0   0   0   0]
 [  5 249   0   0   0   0]
 [  0   0 292   0   0   0]
 [  0   0   0 228   0   0]
 [  0   0   0   0 215   0]
 [  0   0   0   0   0 195]]

 classification_report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       287
           1       1.00      0.98      0.99       254
           2       1.00      1.00      1.00       292
           3       1.00      1.00      1.00       228
           4       1.00      1.00      1.00       215
           5       1.00      1.00      1.00       195

    accuracy                           1.00      1471
   macro avg       1.00      1.00      1.00      1471
weighted avg       1.00      1.00      1.00      1471



In [41]:
# all_hpt_depth_xgb_model 예측결과 저장
## result 데이터프레임 11번 인덱스에 'all_hpt_depth_xgb'(모델명), 'test'(검증 데이터명), accuracy_score 결과, f1_score 결과를 할당해 주세요.
## f1_score의 평균은 'macro' 파라미터를 사용 합니다.
## 예측 결과가 잘 할당 되었는지 확인해 주세요.

result.loc[11] = ['all_hpt_depth_xgb', 'test', accuracy_score(test_y_map,all_hpt_depth_xgb_pred), f1_score(test_y_map,all_hpt_depth_xgb_pred, average = 'macro')]
result

model_name valid_data  accuracy_score  f1_score
0                 svc      train        0.968839  0.971715
1                  lr      train        0.983003  0.984583
2                 knn      train        0.954674  0.958345
3                 xgb      train        0.988669  0.989470
4             hpt_xgb      train        0.990368  0.991052
5         hpt_xgb_150      train        0.990935  0.991562
6          hpt_xgb_50      train        0.985269  0.985278
7         hpt_xgb_top      train        0.991501  0.992040
9    test_hpt_xgb_top       test        0.985724  0.986275
10        all_hpt_xgb       test        0.996601  0.996906
11  all_hpt_depth_xgb       test        0.995921  0.996286

* 결과 판단<BR>
    * 데이터의 학습량을 늘려 과대적합을 해결하려 하였습니다.<BR>
    * 결과를 보고 과대적합이 해소 되었는지 확인 합니다.<BR>

### [미션4] 결과 저장

In [42]:
## 모델 저장하기(코드 수정하지 말고 실행해 주세요)
## 과대적합을 해결한 all_hpt_xgb_model 모델을 joblib.dump 매서드를 사용하여 all_hpt_xgb_model.pkl 파일로 저장 합니다.

joblib.dump(all_hpt_xgb_model, 'all_hpt_xgb_model.pkl')


['all_hpt_xgb_model.pkl']

In [43]:
# 결과 저장
# result 변수를 인덱스 미포함하여 'result5.csv' 파일로 저장 합니다.
# 저장경로는 현재 경로의 하위 './data' 폴더로 지정해 주세요.

result.to_csv('./data/result5.csv', index=False)

In [44]:
# 저장 여부 확인
## 데이터가 파일로 잘 저장 되었는지 result5.csv 파일을 로드하여 확인 합니다.
## 로드한 데이터는 save_check 변수에 할당해 주세요.

save_check = pd.read_csv('./data/result5.csv')
save_check

model_name valid_data  accuracy_score  f1_score
0                 svc      train        0.968839  0.971715
1                  lr      train        0.983003  0.984583
2                 knn      train        0.954674  0.958345
3                 xgb      train        0.988669  0.989470
4             hpt_xgb      train        0.990368  0.991052
5         hpt_xgb_150      train        0.990935  0.991562
6          hpt_xgb_50      train        0.985269  0.985278
7         hpt_xgb_top      train        0.991501  0.992040
8    test_hpt_xgb_top       test        0.985724  0.986275
9         all_hpt_xgb       test        0.996601  0.996906
10  all_hpt_depth_xgb       test        0.995921  0.996286

---------

## 수고하셨습니다

---------